# Importing some packages

In [1]:
import numpy as np
from numba import njit
import pandas as pd
import polars as pl
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# Limited testing

In [2]:
@njit(cache=True)
def get_signals(signal_list, exit_list):

    start_idx = 0
    exit_idx = 0
   
    for i in range(len(signal_list[0])):

        if i == start_idx:

            if signal_list[0][i] == 0:

                start_idx += 1

                exit_list[0][i] = 0

            else:

                for j in range(i+1, len(exit_list[0])):
                    if exit_list[0][j] == -signal_list[0][i]:
                        exit_idx = j
                        break
                    else:
                        exit_idx = j

                for k in range(i+1, exit_idx+1):
                    if signal_list[0][k] == -signal_list[0][i]:
                        exit_idx = k
                        exit_list[1][k] = signal_list[1][k]
                        break
                    else:
                        exit_idx = k

                for p in range(i+1, exit_idx):
                    signal_list[0][p] = 0
                    exit_list[0][p] = 0
                
                if exit_idx == len(signal_list[0]) - 1 and exit_list[0][exit_idx] != -signal_list[0][i]:
                    exit_list[0][exit_idx] = 0
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                else:
                    exit_list[0][exit_idx] = -signal_list[0][i]
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                
                start_idx = exit_idx + 1

        else:

            continue

    if sum(np.abs(signal_list[0])) == sum(np.abs(exit_list[0])):

        return signal_list, exit_list
    
    else:

        for i in range(len(signal_list[0])):
            if signal_list[0][-(i+1)] != 0:
                signal_list[0][-(i+1)] = 0
                break

        return signal_list, exit_list
    
@njit(cache=True)
def create_position_open_prices(signal_list, exit_list):

    pos_open_prices = np.zeros(len(signal_list[0]))
    pos_exit_prices = np.zeros(len(exit_list[0]))

    start_idx = 0
    price_idx = 0

    for i in range(len(signal_list[0])):
        if exit_list[0][i] != 0:
            for j in range(start_idx, i):
                if signal_list[0][j] != 0:
                    price_idx = j
                    break
            pos_open_prices[i] = signal_list[1][price_idx]
            pos_exit_prices[i] = exit_list[1][i]
            start_idx = i
        else:
            pass

    return pos_open_prices, pos_exit_prices

@njit(cache=True)
def get_pnl_testing(
    trade_close_prices,
    signal_list, 
    trade_open_prices,
    commission=0.015,
    slippage=0.05,
    init_inv=20000,
    trade_size=0.1
):

    pnl_list = np.zeros(len(trade_close_prices))

    for i in range(len(trade_close_prices)):

        if signal_list[i] == 0 or trade_open_prices[i] == 0:
            pass
        
        # signal_list contains the points where exit occurs
        elif signal_list[i] == -1: 
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_close_prices[i] - trade_open_prices[i] * (1 + slippage)) 
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        else:
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_open_prices[i] * (1 - slippage) - trade_close_prices[i])
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        pnl_list[i] = temp_pnl

    return pnl_list


## Entry testing

### Fixed stop and target exit

In [3]:
@njit(cache=True)
def get_exit_entry_testing1(
    close_prices, 
    open_prices,  
    signal_list,
    stoploss_th,
    takeprofit_th, 
    commission, 
    slippage, 
    init_inv, 
    trade_size
):

    exit_list = np.zeros((2, len(close_prices)))

    for i in range(len(close_prices)-1):

        if signal_list[1][i] == 0:

            pass

        else:

            temp_n_assets = int(init_inv * trade_size / signal_list[1][i])
            if signal_list[0][i] == 1:
                temp_pnl = temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 + slippage))
            else:
                temp_pnl = -temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 - slippage))
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

            if -temp_pnl >= stoploss_th or temp_pnl >= takeprofit_th:
                exit_list[0][i+1] = -signal_list[0][i]
                exit_list[1][i+1] = open_prices[i+1]
            else:
                pass
        
    return exit_list

### Fixed bar exit

In [4]:
@njit(cache=True)
def get_exit_entry_testing2( 
    open_prices,  
    signal_list,
    n_exit_bars
):

    exit_list = np.zeros((2, len(signal_list[0])))

    n_exit_bars = np.int64(n_exit_bars)

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:
            
            if i + n_exit_bars < len(signal_list[0]):
                exit_list[0][i+n_exit_bars] = -signal_list[0][i]
                exit_list[1][i+n_exit_bars] = open_prices[i+n_exit_bars]
            else:
                pass
        
    return exit_list


### Random exit

In [5]:
@njit(cache=True)
def get_exit_entry_testing3( 
    open_prices,  
    signal_list
):

    exit_list = np.zeros((2, len(signal_list[0])))

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:

            for j in range(i+1, len(signal_list[0])):
                if signal_list[0][j] != 0:
                    j = j - 1
                    break
                else:
                    if np.random.rand() > 0.5:
                        break
            
            exit_list[0][j] = -signal_list[0][i]
            exit_list[1][j] = open_prices[j]
        
    return exit_list


### Winning percentage

In [6]:
def calculate_mean_win_perc_entry_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE'] 

    if len(exec_dict['buy_idxs']) != 0:
        signal_idxs = list(exec_dict['buy_idxs'])
        if len(exec_dict['sell_idxs']) != 0:
            signal_idxs.extend(list(exec_dict['sell_idxs']))
    else:
        if len(exec_dict['sell_idxs']) != 0:
            signal_idxs = list(exec_dict['sell_idxs'])
        else:
            signal_idxs = []

    if len(signal_idxs) != 0:
        signal_idxs = sorted(signal_idxs)
        signal_idxs_true = [i - 1 for i in signal_idxs]
    else:
        signal_idxs_true = []

    df['buy'] = 0
    if len(exec_dict['buy_idxs']) != 0:
        buy_idxs = [i - 1 for i in list(exec_dict['buy_idxs'])]
        df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    df['sell'] = 0
    if len(exec_dict['sell_idxs']) != 0:
        sell_idxs = [i - 1 for i in list(exec_dict['sell_idxs'])]
        df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['new_signal'] = 0
    df['signal_prices'] = 0
    if len(signal_idxs) != 0:
        df.loc[df.index.isin(signal_idxs_true), 'new_signal'] = df.loc[df.index.isin(signal_idxs_true), 'signal'].values
        df.loc[df.index.isin(signal_idxs), 'signal_prices'] = df.loc[df.index.isin(signal_idxs), 'btc_open'].values

    signal_list = np.zeros((2, df.shape[0]))
    signal_list[0][1:] = df['new_signal'].values[:-1]
    signal_list[1] = df['signal_prices'].values

    # fixed stop and target exit testing
    exit_list = get_exit_entry_testing1(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        stoploss_th=50,
        takeprofit_th=100,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # fixed bar exit testing
    exit_list = get_exit_entry_testing2( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list,
        n_exit_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random exit testing
    exit_list = get_exit_entry_testing3( 
        open_prices=df['btc_open'].values,  
        signal_list=signal_list
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    return fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent

def get_entry_win_pc_df(entry_walk_forward_dict, n_not_worked, n_total_cases):
    
    entry_mean_win_perc_dict = defaultdict(list)

    entry_mean_win_perc_dict['Fixed_StopLoss_TakeProfit_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_sp_testing']))
    entry_mean_win_perc_dict['Fixed_Bar_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_bar_testing']))
    entry_mean_win_perc_dict['Random_Exit_testing'].\
        append(np.mean(entry_walk_forward_dict['random_exit_testing']))
    entry_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
        
    entry_win_pc_df = pd.DataFrame(entry_mean_win_perc_dict)

    return entry_win_pc_df


## Exit testing

### Similar approach entry

In [7]:
@njit(cache=True)
def get_entry_exit_testing1(
    close_prices,
    open_prices,
    n_bars
):

    signal_list = np.zeros((2, len(close_prices)))

    # n_bars = np.int64(n_bars)

    for i in range(n_bars, len(signal_list[0])):
            
        if close_prices[i - n_bars] - close_prices[i - 1] > 0:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif close_prices[i - n_bars] - close_prices[i - 1] < 0:
           signal_list[0][i] = -1
           signal_list[1][i] = open_prices[i]
        else:
            pass
        
    return signal_list

@njit(cache=True)
def get_rsi(close_prices, prev_close_prices, length=14):
    # Create numpy arrays to store the gain/loss values
    gains = np.zeros(len(close_prices))
    losses = np.zeros(len(close_prices))

    # Iterate through the data frame and calculate the gain/loss for each period
    for i in range(1, len(close_prices)):
        change = close_prices[i] - prev_close_prices[i]
        if change > 0:
            gains[i] = change
        elif change < 0:
            losses[i] = abs(change)

    # Calculate the average gain and loss for each period
    avg_gains = np.zeros(len(close_prices))
    avg_losses = np.zeros(len(close_prices))
    for i in range(length, len(close_prices)):
        avg_gains[i] = np.mean(gains[i-length:i])
        avg_losses[i] = np.mean(losses[i-length:i])

    # Calculate the relative strength and RSI for each period
    rs = np.zeros(len(close_prices))
    rsi = np.zeros(len(close_prices))
    
    for i in range(len(close_prices)):
        if i+1 < length:
            rsi[i] = -999
        elif avg_losses[i] == 0:
            rs[i] = avg_gains[i]
            rsi[i] = 100
        else:
            rs[i] = avg_gains[i] / avg_losses[i]
            rsi[i] = 100 - (100 / (1 + rs[i]))

    return rsi

@njit(cache=True)
def get_entry_exit_testing2(
    close_prices, open_prices, prev_close_prices, rsi_window_size, rsi_threshold
):

    signal_list = np.zeros((2, len(close_prices)))

    rsi = get_rsi(close_prices, prev_close_prices, length=rsi_window_size)

    for i in range(len(close_prices)-1):

        if i < rsi_window_size - 1 or rsi[i] == -999:
            continue
       
        if rsi[i] < rsi_threshold:
            signal_list[0][i+1] = 1
            signal_list[1][i+1] = open_prices[i+1]
        elif rsi[i] > (100 - rsi_threshold):
            signal_list[0][i+1] = -1
            signal_list[1][i+1] = open_prices[i+1]
        else:
            pass

    return signal_list


### Random entry

In [8]:
@njit(cache=True)
def get_entry_exit_testing3(
    open_prices
):

    signal_list = np.zeros((2, len(open_prices)))

    for i in range(len(open_prices)-1):

        if np.random.rand() > 0.7:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif np.random.rand() < 0.3:
            signal_list[0][i] = -1
            signal_list[1][i] = open_prices[i]
        else:
            pass

    return signal_list


### Winning percentage

In [9]:
def calculate_mean_win_perc_exit_testing(exec_dict, df):

    commission = exec_dict['COMMISSION']
    slippage = exec_dict['SLIPPAGE'] 
    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    if len(exec_dict['sell_exit_idxs']) != 0:
        signal_idxs = list(exec_dict['sell_exit_idxs'])
        if len(exec_dict['buy_exit_idxs']) != 0:
            signal_idxs.extend(list(exec_dict['buy_exit_idxs']))
    else:
        if len(exec_dict['buy_exit_idxs']) != 0:
            signal_idxs = list(exec_dict['buy_exit_idxs'])
        else:
            signal_idxs = []

    if len(signal_idxs) != 0:
        signal_idxs = sorted(signal_idxs)
        signal_idxs_true = [i - 1 for i in signal_idxs]
    else:
        signal_idxs_true = []

    df['buy'] = 0
    if len(exec_dict['sell_exit_idxs']) != 0:
        buy_idxs = [i - 1 for i in list(exec_dict['sell_exit_idxs'])]
        df.loc[df.index.isin(buy_idxs), 'buy'] = 1

    df['sell'] = 0
    if len(exec_dict['buy_exit_idxs']) != 0:
        sell_idxs = [i - 1 for i in list(exec_dict['buy_exit_idxs'])]
        df.loc[df.index.isin(sell_idxs), 'sell'] = -1

    df['signal'] = df['buy'] + df['sell']

    df['exit_signal'] = 0
    df['exit_prices'] = 0
    if len(signal_idxs) != 0:
        df.loc[df.index.isin(signal_idxs_true[1:]), 'exit_signal'] = df.loc[df.index.isin(signal_idxs_true[1:]), 'signal'].values
        df.loc[df.index.isin(signal_idxs[1:]), 'exit_prices'] = df.loc[df.index.isin(signal_idxs[1:]), 'btc_open'].values

    exit_list = np.zeros((2, df.shape[0]))
    exit_list[0][1:] = df['exit_signal'].values[:-1]
    exit_list[1] = df['exit_prices'].values

    # replacing entry with trend following entry
    signal_list = get_entry_exit_testing1(
        close_prices=df['btc_close'].values,
        open_prices=df['btc_open'].values,
        n_bars=5
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    trend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # replacing entry with countertrend entry
    signal_list = get_entry_exit_testing2(
        close_prices=df['btc_close'].values, 
        open_prices=df['btc_open'].values, 
        prev_close_prices=df['btc_close'].shift(1).fillna(method='bfill').values, 
        rsi_window_size=10, 
        rsi_threshold = 20
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    countertrend_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)

    # random entry testing
    signal_list = get_entry_exit_testing3(
        open_prices=df['btc_open'].values
    )

    signal_list, exit_list = get_signals(signal_list, exit_list)
    pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

    pnl_list = get_pnl_testing(
        trade_close_prices=pos_exit_prices,
        signal_list=exit_list[0], 
        trade_open_prices=pos_open_prices,
        commission=commission, 
        slippage=slippage, 
        init_inv=init_inv, 
        trade_size=trade_size
    )

    random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)


    return trend_winning_percent, countertrend_winning_percent, random_winning_percent

def get_exit_win_pc_df(exit_walk_forward_dict, n_not_worked, n_total_cases):

    exit_mean_win_perc_dict = defaultdict(list)

    exit_mean_win_perc_dict['Trend_testing'].\
        append(np.mean(exit_walk_forward_dict['trend_entry_testing']))
    exit_mean_win_perc_dict['Countertrend_testing'].\
        append(np.mean(exit_walk_forward_dict['countertrend_entry_testing']))
    exit_mean_win_perc_dict['Random_Entry_testing'].\
        append(np.mean(exit_walk_forward_dict['random_entry_testing']))
    exit_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    exit_win_pc_df = pd.DataFrame(exit_mean_win_perc_dict)

    return exit_win_pc_df


## Core testing

### Winning percentage

In [10]:
def calculate_mean_win_perc_core_testing(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
    
    return winning_percent

def get_core_win_pc_df(core_walk_forward_dict, n_not_worked, n_total_cases):

    core_mean_win_perc_dict = defaultdict(list)
    core_mean_win_perc_dict['Core_Testing'].\
        append(np.mean(core_walk_forward_dict['core_testing']))
    core_mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    core_win_pc_df = pd.DataFrame(core_mean_win_perc_dict)

    return core_win_pc_df


### Performance

In [11]:
@njit(cache=True)
def get_drawdown(pnl_list):
    max_dd = 0.0
    peak = pnl_list[0]

    for i in range(1, len(pnl_list)):
        if pnl_list[i] > peak:
            peak = pnl_list[i]
        drawdown = 100 * (peak - pnl_list[i]) / peak
        if drawdown > max_dd:
            max_dd = drawdown

    return max_dd

@njit(cache=True)
def get_drawdown_duration(pnl_arr):
    peak = pnl_arr[0]
    max_duration = 0
    current_duration = 0

    for pnl in pnl_arr:
        if pnl < peak:  # We are in a drawdown
            current_duration += 1
        else:  # We found a new peak
            peak = pnl
            current_duration = 0

        max_duration = max(max_duration, current_duration)

    return max_duration

@njit(cache=True)
def get_sharpe_ratio(
    pnl_list, 
    risk_free_rate=0
):
    # mean_return = np.mean(pnl_list)
    mean_return = 0
    for i in range(len(pnl_list)):
        mean_return += pnl_list[i]

    if len(pnl_list) == 0:
        mean_return = 0
    else:
        mean_return = mean_return / len(pnl_list)

    # std_dev = np.std(pnl_list)
    std_dev = 0
    for i in range(len(pnl_list)):
        std_dev += (pnl_list[i] - mean_return) ** 2

    if len(pnl_list) == 1:
        std_dev = np.sqrt(std_dev/len(pnl_list))
    else:
        std_dev = np.sqrt(std_dev/(len(pnl_list) - 1))

    if std_dev == 0:
        std_dev += 0.0001

    sharpe_ratio = (mean_return - risk_free_rate) / std_dev
    return sharpe_ratio

@njit(cache=True)
def get_sortino_ratio(profit_loss, target_pl=0):

    downside_pnl = profit_loss[profit_loss < target_pl]
    if len(downside_pnl) == 0:
        return 0

    downside_deviation = np.sqrt(np.mean((downside_pnl - target_pl) ** 2))
    if downside_deviation == 0:
        return 0

    mean_pnl = np.mean(profit_loss)
    sortino_ratio = (mean_pnl - target_pl) / downside_deviation

    return sortino_ratio


In [12]:
def calculate_mean_performance(exec_dict, monkey_test=False):

    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    signal_idxs = list(exec_dict['buy_idxs'])
    signal_idxs.extend(list(exec_dict['sell_idxs']))
    signal_idxs = sorted(signal_idxs)

    metric_dict = {}

    metric_dict['n_trades'] = len(signal_idxs) - 1
    metric_dict['overall_pnl'] = np.sum(pnl_list)
    metric_dict['roi'] = 100 * metric_dict['overall_pnl'] / (trade_size * init_inv)
    metric_dict['avg_drawdown'] = exec_dict['avg_drawdown']
    metric_dict['max_dd'] = get_drawdown(pnl_list)
    metric_dict['drawdown_dur'] = get_drawdown_duration(pnl_list)
    metric_dict['pnl_avgd_ratio'] = exec_dict['fitness']

    annualized_sharpe_ration = np.sqrt(525600) * get_sharpe_ratio(pnl_list, risk_free_rate=0)
    metric_dict['sharpe_ratio'] = annualized_sharpe_ration

    annualized_sortino_ratio = np.sqrt(525600) * get_sortino_ratio(pnl_list)
    metric_dict['sortino_ratio'] = annualized_sortino_ratio

    if monkey_test:

        pnl_mren_arr = exec_dict['pnl_mren_arr']
        max_dd_mren_arr = exec_dict['max_dd_mren_arr']

        pnl_mren_good_cases = np.sum(np.where(metric_dict['overall_pnl'] > pnl_mren_arr, 1, 0))
        metric_dict['mt_pnl'] = 100 * pnl_mren_good_cases / len(pnl_mren_arr)

        max_dd_mren_good_cases = np.sum(np.where(metric_dict['max_dd'] < max_dd_mren_arr, 1, 0))
        metric_dict['mt_mdd'] = 100 * max_dd_mren_good_cases / len(max_dd_mren_arr)

    return metric_dict

def get_perf_df(performance_walk_forward_dict, n_not_worked, n_total_cases):

    mean_perf_dict = defaultdict(list)
    
    mean_perf_dict['N_Trades'].\
        append(np.mean(performance_walk_forward_dict['n_trades']))
    mean_perf_dict['PNL'].\
        append(np.mean(performance_walk_forward_dict['pnl']))
    mean_perf_dict['ROI (%)'].\
        append(np.mean(performance_walk_forward_dict['roi']))
    mean_perf_dict['AVG_Drawdown'].\
        append(np.mean(performance_walk_forward_dict['avg_drawdown']))
    mean_perf_dict['Drawdown (%)'].\
        append(np.mean(performance_walk_forward_dict['drawdown']))
    mean_perf_dict['Drawdown_Duration'].\
        append(np.mean(performance_walk_forward_dict['drawdown_dur']))
    mean_perf_dict['PNL_AVGD_Ratio'].\
        append(np.mean(performance_walk_forward_dict['pnl_avgd_ratio']))
    mean_perf_dict['Sharpe_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sharpe_ratio']))
    mean_perf_dict['Sortino_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sortino_ratio']))
    mean_perf_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    if 'mt_pnl' in performance_walk_forward_dict.keys():
        mean_perf_dict['MT_PNL_D'].\
            append(np.mean(performance_walk_forward_dict['mt_pnl']))
        mean_perf_dict['MT_MDD_D'].\
            append(np.mean(performance_walk_forward_dict['mt_mdd']))

    perf_df = pd.DataFrame(mean_perf_dict)

    return perf_df


## Equity curve Monte Carlo simulation

In [13]:
@njit(cache=True)
def get_random_idxs4mc(arr, num_elements):
    # Get the length of the input array
    n = len(arr)

    if num_elements > n:
        raise ValueError("num_elements must be less than or equal to the length of the array.")

    # Generate random indices with replacement
    indices = np.random.randint(0, n, num_elements)

    # Return sorted indices
    return indices

@njit(cache=True)
def run_simulation(pnl_list, n_runs, init_inv, trade_size):

    max_dd_array = np.zeros(n_runs)
    dd_dur_array = np.zeros(n_runs)
    profit_array = np.zeros(n_runs)
    roi_array = np.zeros(n_runs)
    binary_profit_array = np.zeros(n_runs, dtype=np.int32)
    
    for i in range(n_runs):
        
        # Generate bootstrap sample
        temp_random_idxs = get_random_idxs4mc(arr=pnl_list, num_elements=len(pnl_list))
        temp_pnl_list = np.zeros(len(temp_random_idxs))
        for j in range(len(temp_random_idxs)):
            temp_pnl_list[j] = pnl_list[temp_random_idxs[j]]

        equity_curve_list = np.cumsum(temp_pnl_list)
        
        # Max Drawdown
        max_dd = get_drawdown(equity_curve_list)
        max_dd_array[i] = max_dd
        
        # Drawdown Duration
        dd_dur = get_drawdown_duration(equity_curve_list)
        dd_dur_array[i] = dd_dur
        
        # Profit
        profit = np.sum(temp_pnl_list)
        profit_array[i] = profit
        
        # ROI
        roi = 100 * profit / (init_inv * trade_size)
        roi_array[i] = roi
        
        # Binary Profit
        if profit > 0:
            binary_profit_array[i] = 1

    return max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array


In [14]:
def get_mc_results(pnl_list, init_inv, trade_size, n_runs):

    max_dd_array, dd_dur_array, profit_array, roi_array, binary_profit_array = run_simulation(pnl_list, n_runs, init_inv, trade_size)

    mc_dict = {}
    mc_dict['median_max_dd'] = np.median(max_dd_array)
    mc_dict['median_dd_dur'] = np.median(dd_dur_array)
    mc_dict['median_profit'] = np.median(profit_array)
    mc_dict['median_return'] = np.median(roi_array)
    mc_dict['return_dd_ratio'] = np.median(roi_array) / np.median(max_dd_array)
    mc_dict['prob_profit'] = np.sum(binary_profit_array) / len(binary_profit_array)

    return mc_dict

In [15]:
def calculate_mc_performance(exec_dict):
            
    pnl_list = exec_dict['all_arr']

    init_inv = exec_dict['AVAILABLE_CAPITAL']
    trade_size = exec_dict['TRADE_SIZE']       

    mc_dict = get_mc_results(pnl_list, init_inv, trade_size, n_runs=10000)

    return mc_dict

def get_mc_df(mc_walk_forward_dict, n_not_worked, n_total_cases):

    mean_mc_dict = defaultdict(list)

    mean_mc_dict['median_drawdown (%)'].\
        append(np.mean(mc_walk_forward_dict['median_max_dd']))
    mean_mc_dict['median_drawdown_duration'].\
        append(np.mean(mc_walk_forward_dict['median_dd_dur']))
    mean_mc_dict['median_profit'].\
        append(np.mean(mc_walk_forward_dict['median_profit']))
    mean_mc_dict['median_ROI (%)'].\
        append(np.mean(mc_walk_forward_dict['median_return']))
    mean_mc_dict['ratio'].\
        append(np.mean(mc_walk_forward_dict['return_dd_ratio']))
    mean_mc_dict['prob'].\
        append(np.mean(mc_walk_forward_dict['prob_profit']))
    mean_mc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)

    mc_df = pd.DataFrame(mean_mc_dict)

    return mc_df


# Data loading function

In [16]:
def generate_52w_data(data_path):
    df = pd.read_csv(data_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    # df = df.iloc[-(7 * 60 * 24 * 52):]
    fold = 2
    df = df.iloc[50400*fold:(fold+1)*50400]
    # df = df.iloc[252000: (252000+50400)]
    # df = df.iloc[(252000+50400):(252000+2*50400)]
    df.sort_values('datetime', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df

In [17]:
# def generate_52w_data(data_path):
    
#     df = pl.read_csv(data_path)
#     df = df.with_columns(pl.col('datetime').str.to_datetime())
#     # df = df.slice(-(7 * 60 * 24 * 52))
#     df = df.sort('datetime', descending=False)

#     return df

# Testing strategies constructed from combination with other instruments

## Loading price and volume data

In [18]:
data_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\all_data_1min_all.csv')
df_52w = generate_52w_data(data_path)
# df_52w = df_52w.iloc[(252000+50400): (252000+2*50400)]
# df_52w.reset_index(drop=True, inplace=True)
df_52w.head()

,datetime,btc_open,btc_high,btc_low,btc_close,btc_volume,6e_open,6e_high,6e_low,6e_close,...,zf_open,zf_high,zf_low,zf_close,zf_volume,zn_open,zn_high,zn_low,zn_close,zn_volume
0,2024-01-10 18:14:00,46286.40,46340.57,46263.62,46275.97,19.135366,1.09950,1.09955,1.09940,1.09950,...,108.140625,108.140625,108.140625,108.140625,1543.0,111.9375,111.9375,111.921875,111.937500,3886.0
1,2024-01-10 18:15:00,46277.74,46284.74,46179.04,46205.95,22.676790,1.09945,1.09950,1.09940,1.09950,...,108.140625,108.148438,108.140625,108.140625,974.0,111.9375,111.9375,111.921875,111.921875,1743.0
2,2024-01-10 18:16:00,46201.64,46227.60,46164.12,46186.87,46.713583,1.09950,1.09980,1.09950,1.09980,...,108.132812,108.140625,108.132812,108.140625,351.0,111.9375,111.9375,111.937500,111.937500,1541.0
3,2024-01-10 18:17:00,46178.58,46300.94,46170.43,46283.52,18.306744,1.09980,1.09980,1.09975,1.09975,...,108.140625,108.140625,108.132812,108.132812,581.0,111.9375,111.9375,111.921875,111.921875,1017.0
4,2024-01-10 18:18:00,46279.20,46300.00,46234.17,46234.17,13.453822,1.09975,1.09985,1.09975,1.09985,...,108.140625,108.140625,108.140625,108.140625,1279.0,111.9375,111.9375,111.921875,111.937500,1776.0


## Loading strategies generated

In [19]:
strategy_file_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\results/\VCh_24_10_29_121047_958800_24976_958800/\ge_results.csv')
strategy_file_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\results/\VCh_24_11_1_120353_214176_28464_214176/\ge_results.csv')
strategy_file_path = Path(r"C:/\Users/\vchar/\Downloads/\ge_results (6).csv")
strategy_file_path = Path(r"C:/\Users/\vchar/\Downloads/\ge_results_crossover6_fold2.csv")

try:
    df_str = pd.read_csv(strategy_file_path)
except:
    df_str = pd.read_csv(strategy_file_path, sep=';')
    
# df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] < -3) & (df_str['fitness'] > -50)]
# df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] <= -1000)] #& (df_str['fitness'] <= -1000)
df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] > -1000)]
df_str = df_str[~df_str.duplicated()]
df_str.sort_values('fitness', ascending=True, inplace=True)
df_str.reset_index(drop=True, inplace=True)
df_str = df_str.iloc[:5000]
df_str

,buy,exit_buy,sell,exit_sell,fitness
0,(numba_indicators.adx(high=price_data['btc_hig...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((numba_indicators.moving_average(prices=price...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,-999.942683
1,(numba_indicators.adx(high=price_data['btc_hig...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((numba_indicators.moving_average(prices=price...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,-999.329466
2,(numba_indicators.adx(high=price_data['btc_hig...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,((numba_indicators.moving_average(prices=price...,((price_data['btc_close'][MAX_LAG:] > get_lag(...,-999.329466
3,((numba_indicators.relative_strength_index(pri...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-996.087560
4,(numba_indicators.ultimate_oscillator(high=pri...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-995.650962
...,...,...,...,...,...
4995,(numba_indicators.ultimate_oscillator(high=pri...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-548.372164
4996,(numba_indicators.ultimate_oscillator(high=pri...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.exponential_moving_average(...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-548.354029
4997,"((get_lag(price_data['btc_high'], lag=3)[MAX_L...",(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.moving_average(prices=price...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-548.349575
4998,(numba_indicators.ultimate_oscillator(high=pri...,(price_data['btc_close'][MAX_LAG:] < signals.m...,((numba_indicators.exponential_moving_average(...,(price_data['btc_close'][MAX_LAG:] < signals.m...,-548.344080


## Running the tests over the strategies

In [ ]:
final_entry_win_pc_df = pd.DataFrame()
final_exit_win_pc_df = pd.DataFrame()
final_core_win_pc_df = pd.DataFrame()
final_perf_df = pd.DataFrame()
final_mc_df = pd.DataFrame()

equity_curve_dict = defaultdict(list)

bars_per_5week = 7 * 60 * 24 * 5
n_bars_per_year = 7 * 60 * 24 * 52

if df_52w.shape[0] < n_bars_per_year:
    n_bars_per_year = df_52w.shape[0]

lag_txt = '{i}'

strategy_idx = 1

for row in tqdm(df_str.itertuples()):#df_str.iloc[:10].itertuples()

    buy_signal_txt = row.buy
    buy_exit_txt = row.exit_buy
    sell_signal_txt = row.sell
    sell_exit_txt = row.exit_sell

    text_code = f'''import os
CUR_DIR = os.getcwd()
os.chdir('src')
#import pandas as pd
import numpy as np
import gc
from fitness.indicators import numba_indicators, signals
from fitness.performance.helper_func import merge_buy_sell_pnl, get_drawdowns, get_pnl, get_lag
from fitness.performance.helper_func import trading_signals_buy, trading_signals_sell, change_exit
os.chdir(CUR_DIR)
#from numba import njit
COMMISSION = 0.015
SLIPPAGE = 0.00005
AVAILABLE_CAPITAL = 700000
TRADE_SIZE = 0.5
MAX_LAG = 99
buy_idxs, buy_exit_idxs = trading_signals_buy(buy_signal={buy_signal_txt}, exit_signal={buy_exit_txt})
sell_idxs, sell_exit_idxs = trading_signals_sell(sell_signal={sell_signal_txt}, exit_signal={sell_exit_txt})
# if (len(buy_idxs) == 0 or len(buy_exit_idxs) == 0) and (len(sell_idxs) == 0 or len(sell_exit_idxs) == 0):
#     fitness = -9999999
#     avg_drawdown = -9999999
# else:
try:
    buy_idxs, buy_exit_idxs, sell_idxs, sell_exit_idxs = change_exit(buy_idxs, buy_exit_idxs, sell_idxs, sell_exit_idxs)
except:
    pass
if (len(buy_idxs) == 0 or len(buy_exit_idxs) == 0) and (len(sell_idxs) == 0 or len(sell_exit_idxs) == 0):
    fitness = -9999999
    avg_drawdown = -9999999
else:
    buy_idxs = np.array(buy_idxs)
    sell_idxs = np.array(sell_idxs)
    open_prices = price_data['btc_open']
    # pnl_mren_arr, max_dd_mren_arr = get_monkey_test_results(open_prices, buy_idxs, sell_idxs, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE)
    buy_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_idxs)]
    buy_exit_prices = open_prices[np.isin(np.arange(len(open_prices)), buy_exit_idxs)]
    sell_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_idxs)]
    sell_exit_prices = open_prices[np.isin(np.arange(len(open_prices)), sell_exit_idxs)]
    buy_arr = get_pnl(buy_exit_prices, buy_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
    buy_pnl = np.sum(buy_arr)
    sell_arr = get_pnl(sell_exit_prices, sell_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
    sell_pnl = np.sum(sell_arr)
    all_arr = merge_buy_sell_pnl(buy_idxs, sell_idxs, buy_arr, sell_arr)
    total_pnl = buy_pnl + sell_pnl
    equity_curve_arr = np.cumsum(all_arr)
    drawdowns = get_drawdowns(equity_curve_arr)
    if len(drawdowns[drawdowns!=0]) != 0:
        avg_drawdown = np.sum(drawdowns[drawdowns!=0]) / len(drawdowns[drawdowns!=0])
        fitness = total_pnl / avg_drawdown
    elif total_pnl <= 0 or len(drawdowns[drawdowns!=0]) == 0:
        fitness = -9999999
        avg_drawdown = -9999999
gc.collect()'''
    
    entry_test_n_not_worked = 0
    exit_test_n_not_worked = 0
    core_test_n_not_worked = 0
    perf_n_not_worked = 0
    mc_n_not_worked = 0
    n_total_cases = 0

    entry_walk_forward_dict = defaultdict(list)

    exit_walk_forward_dict = defaultdict(list)

    core_walk_forward_dict = defaultdict(list)

    performance_walk_forward_dict = defaultdict(list)

    mc_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        # df = df_52w.iloc[idx:idx+bars_per_5week, :]
        # df.reset_index(drop=True, inplace=True)
        df = df_52w.iloc[idx:idx+bars_per_5week, :]
        df.reset_index(drop=True, inplace=True)

        price_data = {}
        for col in df.columns:
            if col == 'datetime':
                continue
            else:
                price_data[col] = df[col].values
        price_data['day_of_week'] = (df['datetime'].dt.dayofweek + 1).values
        price_data['month'] = df['datetime'].dt.month.values

        exec_dict = {'price_data': price_data}
        exec(text_code, exec_dict)

        try:
            equity_curve_arr = exec_dict['equity_curve_arr']
            equity_curve_dict[strategy_idx].append(equity_curve_arr)
        except:
            pass

        try:
            fixed_winning_percent, fixed_bar_winning_percent, random_winning_percent = calculate_mean_win_perc_entry_testing(exec_dict, df)
            entry_walk_forward_dict['fixed_sp_testing'].append(fixed_winning_percent)
            entry_walk_forward_dict['fixed_bar_testing'].append(fixed_bar_winning_percent)
            entry_walk_forward_dict['random_exit_testing'].append(random_winning_percent)
        except:
            entry_test_n_not_worked += 1

        try:
            trend_winning_percent, countertrend_winning_percent, random_winning_percent = calculate_mean_win_perc_exit_testing(exec_dict, df)
            exit_walk_forward_dict['trend_entry_testing'].append(trend_winning_percent)
            exit_walk_forward_dict['countertrend_entry_testing'].append(countertrend_winning_percent)
            exit_walk_forward_dict['random_entry_testing'].append(random_winning_percent)
        except:
            exit_test_n_not_worked += 1

        try:
            winning_percent = calculate_mean_win_perc_core_testing(exec_dict)
            core_walk_forward_dict['core_testing'].append(winning_percent)
        except:
            core_test_n_not_worked += 1

        try:
            metric_dict = calculate_mean_performance(exec_dict, monkey_test=False)
            performance_walk_forward_dict['n_trades'].append(metric_dict['n_trades'])
            performance_walk_forward_dict['pnl'].append(metric_dict['overall_pnl'])
            performance_walk_forward_dict['roi'].append(metric_dict['roi'])
            performance_walk_forward_dict['avg_drawdown'].append(metric_dict['avg_drawdown'])
            performance_walk_forward_dict['drawdown'].append(metric_dict['max_dd'])
            performance_walk_forward_dict['drawdown_dur'].append(metric_dict['drawdown_dur'])
            performance_walk_forward_dict['pnl_avgd_ratio'].append(metric_dict['pnl_avgd_ratio'])
            performance_walk_forward_dict['sharpe_ratio'].append(metric_dict['sharpe_ratio'])
            performance_walk_forward_dict['sortino_ratio'].append(metric_dict['sortino_ratio'])
            if 'mt_pnl' in metric_dict.keys():
                performance_walk_forward_dict['mt_pnl'].append(metric_dict['mt_pnl'])
                performance_walk_forward_dict['mt_mdd'].append(metric_dict['mt_mdd'])
        except:
            perf_n_not_worked += 1

        try:
            mc_dict = calculate_mc_performance(exec_dict)
            mc_walk_forward_dict['median_max_dd'].append(mc_dict['median_max_dd'])
            mc_walk_forward_dict['median_dd_dur'].append(mc_dict['median_dd_dur'])
            mc_walk_forward_dict['median_profit'].append(mc_dict['median_profit'])
            mc_walk_forward_dict['median_return'].append(mc_dict['median_return'])
            mc_walk_forward_dict['return_dd_ratio'].append(mc_dict['return_dd_ratio'])
            mc_walk_forward_dict['prob_profit'].append(mc_dict['prob_profit'])
        except:
            mc_n_not_worked += 1

    temp_signal_df = pd.DataFrame({'strategy': f'strategy{strategy_idx}', 'buy': [buy_signal_txt], 'sell': [sell_signal_txt]})

    entry_win_pc_df = get_entry_win_pc_df(entry_walk_forward_dict, entry_test_n_not_worked, n_total_cases)
    entry_win_pc_df = pd.concat([temp_signal_df, entry_win_pc_df], axis=1)
    final_entry_win_pc_df = pd.concat([final_entry_win_pc_df, entry_win_pc_df])

    exit_win_pc_df = get_exit_win_pc_df(exit_walk_forward_dict, exit_test_n_not_worked, n_total_cases)
    exit_win_pc_df = pd.concat([temp_signal_df, exit_win_pc_df], axis=1)
    final_exit_win_pc_df = pd.concat([final_exit_win_pc_df, exit_win_pc_df])

    core_win_pc_df = get_core_win_pc_df(core_walk_forward_dict, core_test_n_not_worked, n_total_cases)
    core_win_pc_df = pd.concat([temp_signal_df, core_win_pc_df], axis=1)
    final_core_win_pc_df = pd.concat([final_core_win_pc_df, core_win_pc_df])

    perf_df = get_perf_df(performance_walk_forward_dict, perf_n_not_worked, n_total_cases)
    perf_df = pd.concat([temp_signal_df, perf_df], axis=1)
    final_perf_df = pd.concat([final_perf_df, perf_df])

    mc_df = get_mc_df(mc_walk_forward_dict, mc_n_not_worked, n_total_cases)
    mc_df = pd.concat([temp_signal_df, mc_df], axis=1)
    final_mc_df = pd.concat([final_mc_df, mc_df])

    strategy_idx += 1

5000it [1:42:11,  1.23s/it]


In [21]:
final_entry_win_pc_df.reset_index(drop=True, inplace=True)
final_exit_win_pc_df.reset_index(drop=True, inplace=True)
final_core_win_pc_df.reset_index(drop=True, inplace=True) 
final_perf_df.reset_index(drop=True, inplace=True)
final_mc_df.reset_index(drop=True, inplace=True)

## Showing the entry testing results

In [22]:
final_entry_win_pc_df

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
0,strategy1,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,62.410461,55.549382,54.743338,0.0
1,strategy2,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,27.582991,38.439863,38.907078,0.0
2,strategy3,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,27.582991,38.439863,42.183066,0.0
3,strategy4,((numba_indicators.relative_strength_index(pri...,((numba_indicators.moving_average(prices=price...,43.471963,36.099536,35.069548,0.0
4,strategy5,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,45.137699,63.804855,63.526763,0.0
...,...,...,...,...,...,...,...
4995,strategy4996,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,36.745996,52.238125,37.973402,0.0
4996,strategy4997,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.exponential_moving_average(...,27.008198,47.081009,56.277168,0.0
4997,strategy4998,"((get_lag(price_data['btc_high'], lag=3)[MAX_L...",((numba_indicators.moving_average(prices=price...,46.271886,44.010324,48.512100,0.0
4998,strategy4999,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.exponential_moving_average(...,50.536909,50.993509,43.953710,0.0


### Finding strategies that have winning percentage equal to 60 or more for entry testing cases

In [23]:
entry_testing_strategies = final_entry_win_pc_df[
    (final_entry_win_pc_df['Fixed_StopLoss_TakeProfit_testing'] >= 60) & 
    (final_entry_win_pc_df['Fixed_Bar_testing'] >= 60) & 
    (final_entry_win_pc_df['Random_Exit_testing'] >= 60)
]['strategy'].tolist()
final_entry_win_pc_df[final_entry_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
49,strategy50,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,60.303980,72.224868,68.488204,0.0
50,strategy51,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,65.162609,62.050285,65.821378,0.0
51,strategy52,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,65.162609,62.050285,61.567157,0.0
99,strategy100,"((get_lag(price_data['btc_high'], lag=3)[MAX_L...",((numba_indicators.moving_average(prices=price...,65.577480,73.079214,62.371461,0.0
150,strategy151,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,64.438359,63.258786,66.209593,0.0
157,strategy158,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,60.191198,67.271306,71.372738,0.0
158,strategy159,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,60.191198,67.271306,73.200257,0.0
159,strategy160,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,62.274297,72.028416,71.744350,0.0
197,strategy198,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,62.673122,71.788308,69.815851,0.0
309,strategy310,((numba_indicators.relative_strength_index(pri...,((numba_indicators.moving_average(prices=price...,71.431406,66.118245,77.083829,0.0


## Showing exit testing results

In [24]:
final_exit_win_pc_df

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
0,strategy1,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,60.950926,61.541668,47.926258,0.0
1,strategy2,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,60.970770,61.718347,46.960646,0.0
2,strategy3,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,60.970770,61.718347,46.807328,0.0
3,strategy4,((numba_indicators.relative_strength_index(pri...,((numba_indicators.moving_average(prices=price...,61.058083,61.716362,47.509723,0.0
4,strategy5,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,61.036255,61.591297,47.245813,0.0
...,...,...,...,...,...,...,...
4995,strategy4996,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,61.081896,61.521817,47.631803,0.0
4996,strategy4997,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.exponential_moving_average(...,61.004505,61.638941,46.723149,0.0
4997,strategy4998,"((get_lag(price_data['btc_high'], lag=3)[MAX_L...",((numba_indicators.moving_average(prices=price...,60.915206,61.511891,47.370614,0.0
4998,strategy4999,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.exponential_moving_average(...,60.946957,61.623060,47.472763,0.0


### Showing strategies that have passed entry testing

In [25]:
final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(entry_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
49,strategy50,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,60.429028,61.015603,46.395619,0.0
50,strategy51,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,60.931082,61.662763,47.237048,0.0
51,strategy52,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,60.931082,61.662763,46.918284,0.0
99,strategy100,"((get_lag(price_data['btc_high'], lag=3)[MAX_L...",((numba_indicators.moving_average(prices=price...,61.014427,61.553579,47.539390,0.0
150,strategy151,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,60.962832,61.601223,47.795547,0.0
157,strategy158,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,60.593733,61.627030,46.210934,0.0
158,strategy159,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,60.593733,61.627030,47.713923,0.0
159,strategy160,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,60.442919,61.073173,47.294589,0.0
197,strategy198,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,60.327823,61.192282,48.257867,0.0
309,strategy310,((numba_indicators.relative_strength_index(pri...,((numba_indicators.moving_average(prices=price...,61.068005,61.557550,46.148504,0.0


### Showing strategies that have winning percentage equal to 60 or more (except for Random_Entry_testing)

In [26]:
exit_testing_strategies = final_exit_win_pc_df[
    (final_exit_win_pc_df['Trend_testing'] >= 60) & 
    (final_exit_win_pc_df['Countertrend_testing'] >= 60) & 
    (final_exit_win_pc_df['Random_Entry_testing'] >= 50) &
    (final_exit_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_exit_win_pc_df[final_exit_win_pc_df['strategy'].isin(exit_testing_strategies)]

,strategy,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working


## Showing core testing results

In [27]:
final_core_win_pc_df

,strategy,buy,sell,Core_Testing,Not_Working
0,strategy1,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,69.729730,0.0
1,strategy2,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,79.054054,0.0
2,strategy3,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,79.054054,0.0
3,strategy4,((numba_indicators.relative_strength_index(pri...,((numba_indicators.moving_average(prices=price...,58.490566,0.0
4,strategy5,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,64.361702,0.0
...,...,...,...,...,...
4995,strategy4996,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,60.337553,0.0
4996,strategy4997,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.exponential_moving_average(...,76.785714,0.0
4997,strategy4998,"((get_lag(price_data['btc_high'], lag=3)[MAX_L...",((numba_indicators.moving_average(prices=price...,71.621622,0.0
4998,strategy4999,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.exponential_moving_average(...,64.161850,0.0


### Showing strategies that has passed entry testing and have winning percentage equal to 60 or more

In [28]:
core_testing_strategies = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(entry_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies)]

,strategy,buy,sell,Core_Testing,Not_Working
50,strategy51,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,72.139303,0.0
51,strategy52,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,72.139303,0.0
99,strategy100,"((get_lag(price_data['btc_high'], lag=3)[MAX_L...",((numba_indicators.moving_average(prices=price...,65.269461,0.0
150,strategy151,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,75.647668,0.0
324,strategy325,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,74.111675,0.0
325,strategy326,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,74.111675,0.0
433,strategy434,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,67.961165,0.0
483,strategy484,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,88.333333,0.0
484,strategy485,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,88.333333,0.0
485,strategy486,((numba_indicators.relative_strength_index(pri...,((numba_indicators.moving_average(prices=price...,63.414634,0.0


### Showing strategies that has passed both entry and exit testing and have winning percentage equal to 60 or more

In [29]:
core_testing_strategies2 = final_core_win_pc_df[
    (final_core_win_pc_df['Core_Testing'] >= 60) &
    (final_core_win_pc_df['strategy'].isin(exit_testing_strategies))
]['strategy'].tolist()

final_core_win_pc_df[final_core_win_pc_df['strategy'].isin(core_testing_strategies2)]

,strategy,buy,sell,Core_Testing,Not_Working


## Showing performance

In [30]:
final_perf_df

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working
0,strategy1,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,184.0,92574.055176,26.449730,579.720470,238.976323,176.0,159.687401,316.161931,408.786023,0.0
1,strategy2,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,147.0,189117.447772,54.033557,564.998622,340.469566,131.0,334.721963,401.694833,896.889785,0.0
2,strategy3,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,147.0,189117.447772,54.033557,564.998622,340.469566,131.0,334.721963,401.694833,896.889785,0.0
3,strategy4,((numba_indicators.relative_strength_index(pri...,((numba_indicators.moving_average(prices=price...,52.0,5834.884902,1.667110,1420.323941,190.626908,14.0,4.108137,16.179646,15.885122,0.0
4,strategy5,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,187.0,253417.457733,72.404988,928.376479,140.161170,172.0,272.968417,310.463299,1203.130559,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,strategy4996,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,236.0,218401.065066,62.400304,543.630491,112.456543,106.0,401.745430,215.576951,1559.875525,0.0
4996,strategy4997,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.exponential_moving_average(...,111.0,264124.083326,75.464024,520.384521,122.911554,53.0,507.555611,394.385600,2860.004942,0.0
4997,strategy4998,"((get_lag(price_data['btc_high'], lag=3)[MAX_L...",((numba_indicators.moving_average(prices=price...,147.0,195974.979768,55.992851,816.972332,128.724727,66.0,239.879580,283.695936,796.649267,0.0
4998,strategy4999,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.exponential_moving_average(...,172.0,264836.530460,75.667580,864.929133,115.362102,67.0,306.194485,305.087041,1470.399377,0.0


### Showing performance for strategies that has passed entry and core testing and have positive ROI

In [31]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working
50,strategy51,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,200.0,119436.324940,34.124664,807.285654,178.802384,176.0,147.948033,287.572370,380.518245,0.0
51,strategy52,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,200.0,119436.324940,34.124664,807.285654,178.802384,176.0,147.948033,287.572370,380.518245,0.0
99,strategy100,"((get_lag(price_data['btc_high'], lag=3)[MAX_L...",((numba_indicators.moving_average(prices=price...,166.0,122247.429128,34.927837,1148.250361,246.955823,86.0,106.464089,202.642638,347.944911,0.0
150,strategy151,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,192.0,143902.050610,41.114872,605.744828,178.802384,168.0,237.562161,363.741117,539.678993,0.0
324,strategy325,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,196.0,144238.424772,41.210979,584.204036,178.802384,172.0,246.897344,353.428988,547.259074,0.0
325,strategy326,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,196.0,144238.424772,41.210979,584.204036,178.802384,172.0,246.897344,353.428988,547.259074,0.0
433,strategy434,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,205.0,105728.117366,30.208034,850.489860,842.906461,182.0,124.314377,252.280230,364.333036,0.0
483,strategy484,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,59.0,171939.494792,49.125570,894.240846,462.416174,26.0,192.274258,451.963828,2013.204919,0.0
484,strategy485,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,59.0,172869.055452,49.391159,816.779394,251.193273,26.0,211.647180,455.552158,2293.016999,0.0
485,strategy486,((numba_indicators.relative_strength_index(pri...,((numba_indicators.moving_average(prices=price...,40.0,11102.426590,3.172122,1785.462537,175.737314,11.0,6.218236,34.279311,29.973574,0.0


### Showing performance for strategies that has passed all tests and have positive ROI

In [32]:
final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]

,strategy,buy,sell,N_Trades,PNL,ROI (%),AVG_Drawdown,Drawdown (%),Drawdown_Duration,PNL_AVGD_Ratio,Sharpe_Ratio,Sortino_Ratio,Not_Working


In [33]:
# final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ][[col for col in final_perf_df.columns if col not in ['buy', 'sell']]]

In [34]:
# buy_str_list = final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ]['buy'].tolist()

# print(buy_str_list[3])

In [35]:
# sell_str_list = final_perf_df[
#     (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
#     (final_perf_df['ROI (%)'] > 0)
# ]['sell'].tolist()

# print(sell_str_list[3])

### Showing Monte Carlo simulation

In [36]:
final_mc_df

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
0,strategy1,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,38.741378,13.0,92496.967525,26.427705,0.682157,1.0000,0.0
1,strategy2,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,14.488603,6.0,187654.530037,53.615580,3.700535,1.0000,0.0
2,strategy3,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,14.157937,6.0,188169.124678,53.762607,3.797348,1.0000,0.0
3,strategy4,((numba_indicators.relative_strength_index(pri...,((numba_indicators.moving_average(prices=price...,315.969444,30.0,5265.811247,1.504517,0.004762,0.5591,0.0
4,strategy5,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,25.272792,11.0,252072.600468,72.020743,2.849734,1.0000,0.0
...,...,...,...,...,...,...,...,...,...,...
4995,strategy4996,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,31.671202,16.0,215106.431457,61.458980,1.940532,1.0000,0.0
4996,strategy4997,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.exponential_moving_average(...,6.374437,4.0,261775.555964,74.793016,11.733273,1.0000,0.0
4997,strategy4998,"((get_lag(price_data['btc_high'], lag=3)[MAX_L...",((numba_indicators.moving_average(prices=price...,22.584406,10.0,193559.563938,55.302733,2.448713,1.0000,0.0
4998,strategy4999,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.exponential_moving_average(...,22.073876,10.0,262809.261560,75.088360,3.401684,1.0000,0.0


### Showing simulation results for strategies that has passed entry and core testing and have positive ROI

In [37]:
mc_strategies_list = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy']

final_mc_df[final_mc_df['strategy'].isin(mc_strategies_list)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working
50,strategy51,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,42.507900,15.0,119120.374166,34.034393,0.800660,1.0000,0.0
51,strategy52,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,41.716056,15.0,118782.924215,33.937978,0.813547,1.0000,0.0
99,strategy100,"((get_lag(price_data['btc_high'], lag=3)[MAX_L...",((numba_indicators.moving_average(prices=price...,57.455596,21.0,120965.852165,34.561672,0.601537,1.0000,0.0
150,strategy151,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,25.000182,10.0,143375.713775,40.964490,1.638568,1.0000,0.0
324,strategy325,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,27.169363,10.0,143465.853400,40.990244,1.508694,1.0000,0.0
325,strategy326,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,26.840576,10.0,144032.793591,41.152227,1.533210,1.0000,0.0
433,strategy434,(numba_indicators.adx(high=price_data['btc_hig...,((numba_indicators.moving_average(prices=price...,54.408307,18.0,105549.018682,30.156862,0.554269,1.0000,0.0
483,strategy484,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,4.783062,3.0,169948.035539,48.556582,10.151777,1.0000,0.0
484,strategy485,(numba_indicators.ultimate_oscillator(high=pri...,((numba_indicators.moving_average(prices=price...,4.309200,3.0,170866.994453,48.819141,11.329050,1.0000,0.0
485,strategy486,((numba_indicators.relative_strength_index(pri...,((numba_indicators.moving_average(prices=price...,216.814910,20.0,10947.288764,3.127797,0.014426,0.6262,0.0


### Showing simulation results for strategies that has passed all tests and have positive ROI

In [38]:
mc_strategies_list2 = final_perf_df[
    (final_perf_df['strategy'].isin(core_testing_strategies2)) & 
    (final_perf_df['ROI (%)'] > 0)
]['strategy']

final_mc_df[final_mc_df['strategy'].isin(mc_strategies_list2)]

,strategy,buy,sell,median_drawdown (%),median_drawdown_duration,median_profit,median_ROI (%),ratio,prob,Not_Working


In [39]:
final_entry_win_pc_df.to_csv('testing_results/entry_testing_fold2_p2_crossover6.csv', index=False)
final_exit_win_pc_df.to_csv('testing_results/exit_testing_fold2_p2_crossover6.csv', index=False)
final_core_win_pc_df.to_csv('testing_results/core_testing_fold2_p2_crossover6.csv', index=False)
final_perf_df.to_csv('testing_results/perf_fold2_p2_crossover6.csv', index=False)
final_mc_df.to_csv('testing_results/mc_fold2_p2_crossover6.csv', index=False)